In [1]:
import carla
client = carla.Client('localhost', 2000)
client.load_world('Town05') #This may give an error "RuntimeError: time-out of 5000ms while w..." - IGNORE

RuntimeError: time-out of 5000ms while waiting for the simulator, make sure the simulator is ready and connected to localhost:2000

In [2]:
#get world object and select preferred car
world = client.get_world()
vehicle_bp = world.get_blueprint_library().filter('*model3*')

In [10]:
#confirm only one car
len(vehicle_bp)

1

In [3]:
#get all highway spawn points for town 5
import random
good_roads = [12, 34, 35, 36, 37, 38, 1201, 1236, 2034, 2035, 2343, 2344]
spawn_points = world.get_map().get_spawn_points()
good_spawn_points = []
for point in spawn_points:
    this_waypoint = world.get_map().get_waypoint(point.location,project_to_road=True, lane_type=(carla.LaneType.Driving))
    if this_waypoint.road_id in good_roads:
        good_spawn_points.append(point)

transform = random.choice(good_spawn_points)

In [19]:
print(transform) # location of spawning point

Transform(Location(x=7.056836, y=-207.482941, z=0.300000), Rotation(pitch=0.767256, yaw=179.757080, roll=0.000000))


In [4]:
#spawn the car
vehicle = world.try_spawn_actor(vehicle_bp[0], transform)

In [5]:
print(vehicle.get_transform()) 

Transform(Location(x=69.430626, y=-186.747208, z=0.001650), Rotation(pitch=0.000000, yaw=-0.242950, roll=0.000000))


In [24]:
#example of getting straight direction - values low, likely due to the car being straight
print(vehicle.get_transform().get_forward_vector())

Vector3D(x=0.999928, y=-0.004226, z=-0.011186)


In [55]:
#spin the car a little - 30 degrees right
trans = vehicle.get_transform()
trans.rotation.yaw = trans.rotation.yaw + 30
vehicle.set_transform(trans)

In [26]:
# now get straight vector after spinning 30 degrees right - strangely both x & y changed and none by 30 degree
print(vehicle.get_transform().get_forward_vector())

Vector3D(x=0.868084, y=0.496311, z=-0.010304)


In [34]:
import numpy as np
import math
print("\nResult...",math.degrees(np.arctanh(0.496311/0.868084)))


Result... 37.24749732679996


In [35]:
#spin the car back - 30 degrees
trans = vehicle.get_transform()
trans.rotation.yaw = trans.rotation.yaw - 30
vehicle.set_transform(trans)

In [36]:
print(vehicle.get_transform().get_forward_vector())

Vector3D(x=0.999930, y=-0.004226, z=-0.011083)


In [37]:
#spin the car 90 degrees right
trans = vehicle.get_transform()
trans.rotation.yaw = trans.rotation.yaw + 90
vehicle.set_transform(trans)

In [38]:
print(vehicle.get_transform().get_forward_vector())

Vector3D(x=0.004302, y=0.999991, z=-0.000274)


In [43]:
0.496311/(0.868084+0.496311) * 90

32.738312585431636

In [44]:
#spin the car straight
trans = vehicle.get_transform()
trans.rotation.yaw = trans.rotation.yaw - 90
vehicle.set_transform(trans)

Vector3D(x=0.710117, y=0.704029, z=-0.008788)


In [49]:
0.49/(0.86+0.49) * 90


32.666666666666664

In [58]:
vector = vehicle.get_transform().get_forward_vector()
x = vector.x
y = vector.y
print("Rough angle:", y/(x+y))

Rough angle: 0.36371525400034294


In [59]:
#spin the car back - 30 degrees
trans = vehicle.get_transform()
trans.rotation.yaw = trans.rotation.yaw - 30
vehicle.set_transform(trans)

In [60]:
vector = vehicle.get_transform().get_forward_vector()
x = vector.x
y = vector.y
print("Rough angle:", y/(x+y))

Rough angle: -0.004325520234127645


In [61]:
#spin the car back - 30 degrees
trans = vehicle.get_transform()
trans.rotation.yaw = trans.rotation.yaw - 30
vehicle.set_transform(trans)

In [67]:
vector = vehicle.get_transform().get_forward_vector()
x = vector.x
y = vector.y
print("Rough angle:", y/(abs(x)+abs(y)))

Rough angle: -0.3683309051529466


In [64]:
y

-0.5036982893943787

In [65]:
print(vehicle.get_transform().get_forward_vector())

Vector3D(x=0.863817, y=-0.503698, z=-0.010367)


In [10]:
vehicle.set_autopilot(True)


In [11]:
vehicle.set_autopilot(False)

In [12]:
vehicle.destroy()

True

In [5]:
current_waypoint = world.get_map().get_waypoint(vehicle.get_location(),project_to_road=True, lane_type=(carla.LaneType.Driving | carla.LaneType.Sidewalk))

In [8]:
print(current_waypoint.transform.rotation.yaw - vehicle.get_transform().rotation.yaw)

360.0000304877758


In [7]:
print(vehicle.get_transform().rotation.yaw)

-0.2429504096508026


In [3]:
# trying to get all waypoints
world = client.get_world()
# good road_ids
good_roads = [12, 34, 35, 36, 37, 38, 1201, 1236, 2034, 2035, 2343, 2344]
town_map = world.get_map()
all_waypoint_pairs = town_map.get_topology()
# subset of lane start/end's which belong to good roads
good_lanes = []
for w in all_waypoint_pairs:
    if w[0].road_id in good_roads:
        good_lanes.append(w)
        

In [4]:
# loop through selected lanes - trace each lane
# this is using the spectator
import time
spectator = world.get_spectator()
for lane in good_lanes:
    #set start and end of the lane
    for wp in lane[0].next_until_lane_end(10):
        transform = wp.transform
        #lift above the road by 1m
        transform.location.z = transform.location.z+1
        spectator.set_transform(transform)
        print(wp.road_id)
        time.sleep(0.01)
        

    



2034
2034
2034
2034
2034
2034
2034
2034
2034
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
36
36
36
36
36
36
36
36
36
36
36
36
36
36
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
3

In [17]:
import time as tm
import random
transform = random.choice(world.get_map().get_spawn_points())

vehicle = world.try_spawn_actor(vehicle_bp[0], transform)
spectator = world.get_spectator()
for lane in good_lanes:
    #loop within a lane
    for wp in lane[0].next_until_lane_end(20):
        transform = wp.transform
        vehicle.set_transform(transform)
        # do multiple shots of straight direction
        spec_trans = transform
        spec_trans.location.z = spec_trans.location.z+2
        spectator.set_transform(spec_trans)
        for i in range(5):
            trans = wp.transform
            angle_adj = random.randrange(-40, 40, 1)
            print('initial angle in this point: ',transform.rotation.yaw, " angle applied: ",angle_adj)
            trans.rotation.yaw = transform.rotation.yaw +angle_adj 
            vehicle.set_transform(trans)
            
            # move the car to location
            vehicle.set_transform(transform)
            tm.sleep(1)

initial angle in this point:  359.757080078125  angle applied:  -14
initial angle in this point:  359.757080078125  angle applied:  11
initial angle in this point:  359.757080078125  angle applied:  -34
initial angle in this point:  359.757080078125  angle applied:  13
initial angle in this point:  359.757080078125  angle applied:  -32
initial angle in this point:  359.757080078125  angle applied:  -10
initial angle in this point:  359.757080078125  angle applied:  -22
initial angle in this point:  359.757080078125  angle applied:  39
initial angle in this point:  359.757080078125  angle applied:  -3
initial angle in this point:  359.757080078125  angle applied:  -10
initial angle in this point:  359.757080078125  angle applied:  -31
initial angle in this point:  359.757080078125  angle applied:  32
initial angle in this point:  359.757080078125  angle applied:  3
initial angle in this point:  359.757080078125  angle applied:  -3
initial angle in this point:  359.757080078125  angle ap

KeyboardInterrupt: 

In [3]:
import random
    

In [6]:
'_out_ang/%06d.png' % tm.time_ns()

'_out_ang/1676272849001680300.png'

In [2]:
angle = 30

In [10]:
'_out_ang/%06d.png' % tm.time_ns() angle

SyntaxError: invalid syntax (386112919.py, line 1)

In [4]:
'_out_ang/%06d_%s.png' % (tm.time_ns(), angle)

'_out_ang/1676273603404611700_30.png'

In [1]:
#from CARLA camera tutorial on YouTube
# this approach make the camera image available with for a simple loop

import carla
import math
import time
import cv2
import numpy as np
import random

client = carla.Client('localhost', 2000)
client.load_world('Town05') #This may give an error "RuntimeError: time-out of 5000ms while w..." - IGNORE

In [2]:
#get world object and select preferred car
world = client.get_world()
bp_lib = world.get_blueprint_library()
vehicle_bp = bp_lib.filter('*model3*')
transform = random.choice(world.get_map().get_spawn_points())
vehicle = world.try_spawn_actor(vehicle_bp[0], transform)

camera_bp = bp_lib.find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640') # this ratio works in CARLA 9.14 on Windows
camera_bp.set_attribute('image_size_y', '360')

camera_init_trans = carla.Transform(carla.Location(z=1.6,x=0.2))
camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)

def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h,image_w,4))}
camera.listen(lambda image: camera_callback(image,camera_data))

cv2.namedWindow('RGB Camera',cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera',camera_data['image'])
cv2.waitKey(1)

vehicle.set_autopilot(True)

#main loop until letter Q is pressed
while True:
    cv2.imshow('RGB Camera',camera_data['image'])
    if cv2.waitKey(1) == ord('q'):
        break
cv2.destroyAllWindows()
camera.stop()
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()

In [5]:
#this it to switch to synchro mode

# this was used to find x=0.9 for camera mount point works on Model 3

PREFERRED_SPEED = 10

world = client.get_world()

traffic_manager = client.get_trafficmanager(8000)
settings = world.get_settings()
traffic_manager.set_synchronous_mode(True)
# option preferred speed
# traffic_manager.set_desired_speed(vehicle,float(PREFERRED_SPEED))
settings.synchronous_mode = True
settings.fixed_delta_seconds = 0.05
world.apply_settings(settings)

bp_lib = world.get_blueprint_library()
vehicle_bp = bp_lib.filter('*model3*')
transform = random.choice(world.get_map().get_spawn_points())
vehicle = world.try_spawn_actor(vehicle_bp[0], transform)

camera_bp = bp_lib.find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640') # this ratio works in CARLA 9.14 on Windows
camera_bp.set_attribute('image_size_y', '360')

camera_init_trans = carla.Transform(carla.Location(z=1.6,x=0.9))
camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)

def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h,image_w,4))}
camera.listen(lambda image: camera_callback(image,camera_data))

cv2.namedWindow('RGB Camera',cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera',camera_data['image'])
cv2.waitKey(1)

#optional autopilot
vehicle.set_autopilot(True)

#main loop until letter Q is pressed
while True:
    cv2.imshow('RGB Camera',camera_data['image'])
    world.tick()
    if cv2.waitKey(1) == ord('q'):
        break
        
#clean up
cv2.destroyAllWindows()
camera.stop()
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()

: 

In [2]:
#transform car through waypoints in a loop

YAW_ADJ_DEGREES = 35
PREFERRED_SPEED = 10
good_roads = [12, 34, 35, 36, 37, 38, 1201, 1236, 2034, 2035, 2343, 2344]

world = client.get_world()

traffic_manager = client.get_trafficmanager(8000)
settings = world.get_settings()
traffic_manager.set_synchronous_mode(True)
# option preferred speed
# traffic_manager.set_desired_speed(vehicle,float(PREFERRED_SPEED))
settings.synchronous_mode = True
settings.fixed_delta_seconds = 0.05
world.apply_settings(settings)

bp_lib = world.get_blueprint_library()
vehicle_bp = bp_lib.filter('*model3*')

town_map = world.get_map()

# optional - get all intersections to avoid cutting images near them
junction_list = []
waypoint_list = town_map.generate_waypoints(2.0)
for x in waypoint_list:
    if x.get_junction() is not None:
        junction_list.append(x.transform.location)
                
#limit spawn points to highways
spawn_points = town_map.get_spawn_points()
good_spawn_points = []
for point in spawn_points:
    this_waypoint = town_map.get_waypoint(point.location,project_to_road=True, lane_type=(carla.LaneType.Driving))
    if this_waypoint.road_id in good_roads:
        good_spawn_points.append(point)

all_waypoint_pairs = town_map.get_topology()
# subset of lane start/end's which belong to good roads
good_lanes = []
for w in all_waypoint_pairs:
    if w[0].road_id in good_roads:
        good_lanes.append(w)        

transform = random.choice(good_spawn_points)
vehicle = world.try_spawn_actor(vehicle_bp[0], transform)

#setting RGB Camera
camera_bp = bp_lib.find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640') # this ratio works in CARLA 9.14 on Windows
camera_bp.set_attribute('image_size_y', '360')

camera_init_trans = carla.Transform(carla.Location(z=1.6,x=0.4))
camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)

def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h,image_w,4))}
camera.listen(lambda image: camera_callback(image,camera_data))

cv2.namedWindow('RGB Camera',cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera',camera_data['image'])
cv2.waitKey(1)

#optional autopilot
vehicle.set_autopilot(True)

#main loop 

for lane in good_lanes:
    #loop within a lane
    for wp in lane[0].next_until_lane_end(20):
        transform = wp.transform
        vehicle.set_transform(transform)
        # do multiple shots of straight direction
        for i in range(5):
            trans = wp.transform
            angle_adj = random.randrange(-YAW_ADJ_DEGREES, YAW_ADJ_DEGREES, 1)
            trans.rotation.yaw = transform.rotation.yaw +angle_adj 
            vehicle.set_transform(trans)
            # Carla Tick
            world.tick()
            cv2.imshow('RGB Camera',camera_data['image'])
            cv2.waitKey(1)
            # move the car to location
            vehicle.set_transform(transform)

#clean up
cv2.destroyAllWindows()
camera.stop()
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()

In [2]:
#transform car through waypoints in a loop while printing the angle onto the image

YAW_ADJ_DEGREES = 35
PREFERRED_SPEED = 10
good_roads = [12, 34, 35, 36, 37, 38, 1201, 1236, 2034, 2035, 2343, 2344]

world = client.get_world()

traffic_manager = client.get_trafficmanager(8000)
settings = world.get_settings()
traffic_manager.set_synchronous_mode(True)
# option preferred speed
# traffic_manager.set_desired_speed(vehicle,float(PREFERRED_SPEED))
settings.synchronous_mode = True
settings.fixed_delta_seconds = 0.05
world.apply_settings(settings)

bp_lib = world.get_blueprint_library()
vehicle_bp = bp_lib.filter('*model3*')

town_map = world.get_map()

# optional - get all intersections to avoid cutting images near them
junction_list = []
waypoint_list = town_map.generate_waypoints(2.0)
for x in waypoint_list:
    if x.get_junction() is not None:
        junction_list.append(x.transform.location)
                
#limit spawn points to highways
spawn_points = town_map.get_spawn_points()
good_spawn_points = []
for point in spawn_points:
    this_waypoint = town_map.get_waypoint(point.location,project_to_road=True, lane_type=(carla.LaneType.Driving))
    if this_waypoint.road_id in good_roads:
        good_spawn_points.append(point)

all_waypoint_pairs = town_map.get_topology()
# subset of lane start/end's which belong to good roads
good_lanes = []
for w in all_waypoint_pairs:
    if w[0].road_id in good_roads:
        good_lanes.append(w)        

transform = random.choice(good_spawn_points)
vehicle = world.try_spawn_actor(vehicle_bp[0], transform)

#setting RGB Camera
camera_bp = bp_lib.find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640') # this ratio works in CARLA 9.14 on Windows
camera_bp.set_attribute('image_size_y', '360')

camera_init_trans = carla.Transform(carla.Location(z=1.6,x=0.4))
camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)

def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h,image_w,4))}
camera.listen(lambda image: camera_callback(image,camera_data))

cv2.namedWindow('RGB Camera',cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera',camera_data['image'])
cv2.waitKey(1)

#optional autopilot
vehicle.set_autopilot(True)

#adding params to display text to image
font = cv2.FONT_HERSHEY_SIMPLEX
# org
org = (50, 50)
fontScale = 1
# white color
color = (255, 255, 255)
# Line thickness of 2 px
thickness = 2

#main loop 
quit = False

for lane in good_lanes:
    #loop within a lane
    if quit:
        break
    for wp in lane[0].next_until_lane_end(20):
        transform = wp.transform
        vehicle.set_transform(wp.transform)
        time.sleep(2)
        initial_yaw = wp.transform.rotation.yaw
        # do multiple shots of straight direction
        for i in range(5):
            # Carla Tick
            world.tick()
            
            trans = wp.transform
            angle_adj = random.randrange(-YAW_ADJ_DEGREES, YAW_ADJ_DEGREES, 1)
            trans.rotation.yaw = initial_yaw +angle_adj 
            vehicle.set_transform(trans)
            time.sleep(1)
            if cv2.waitKey(1) == ord('q'):
                quit = True
                break
            img = camera_data['image']
            # Using cv2.putText() method
            actual_angle = vehicle.get_transform().rotation.yaw - initial_yaw
            if actual_angle <-180:
                actual_angle +=360
            elif actual_angle >180:
                actual_angle -=360
            img = cv2.putText(img, str(actual_angle), org, font, fontScale, color, thickness, cv2.LINE_AA)
            cv2.imshow('RGB Camera',img)
            
#clean up
cv2.destroyAllWindows()
camera.stop()
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()

In [17]:
# finding a particular location
# pick road ids one by one and see where they are

world = client.get_world()
#good_roads = [12, 34, 35, 36, 37, 38, 1201, 1236, 2034, 2035, 2343, 2344]
# road 34 has a bend in it
# road 37 has 2 bends, uphill and downhil sections
# road 38 is just long bend
# the rest are very short and insignificant
good_roads = [2344]

town_map = world.get_map()
all_waypoint_pairs = town_map.get_topology()
# subset of lane start/end's which belong to good roads
good_lanes = []
for w in all_waypoint_pairs:
    if w[0].road_id in good_roads:
        good_lanes.append(w)
# loop through selected lanes - trace each lane
# this is using the spectator
import time
spectator = world.get_spectator()
for lane in good_lanes:
    #set start and end of the lane
    for wp in lane[0].next_until_lane_end(10):
        transform = wp.transform
        #lift above the road by 1m
        transform.location.z = transform.location.z+1
        spectator.set_transform(transform)
        #print(wp.road_id)
        time.sleep(0.2)